In [ ]:
reload_lamb()

## Metalanguage strategy 1: multidimensional types

This is roughly adapted from Potts' dissertation.  The basic idea is to model directly in the metalanguage the side-computation of presuppositions in tandem with the ordinary meaning.  This requires two adjustments:
* lift all instances of type `t` to be type `(t,t)`.  There is no easy and simple way to do this automatically.
* allow in general for types to be either `X` or `(X,t)` for arbitrary `X`.  This is a bit easier to generalize.

A consequence of this strategy is that any item that interacts with functional types must be completely explicit about what happens with presuppositions from its arguments.

Some sample presupposition items:

In [1]:
%%lamb
||john|| = (J_e, Salient(J_e))
||doctor|| = L x_e : (Doctor(x), Human(x))
||saw|| = L x_e : L y_e : (Saw(y,x), PhysExistence(x) & HasVisualSystem(y))
||the|| = L f_<e,(t,t)> : ((Iota x_e : f(x)[0] & f(x)[1]), (ExistsExact x_e : f(x)[0] & f(x)[1]))

INFO (meta): Coerced guessed type for 'Salient_t' into <e,t>, to match argument 'J_e'
INFO (meta): Coerced guessed type for 'Doctor_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Human_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'
INFO (meta): Coerced guessed type for 'PhysExistence_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'HasVisualSystem_t' into <e,t>, to match argument 'y_e'


$[\![\mathbf{\text{john}}]\!]^{}_{\left(e, t\right)} \:=\: $$({J}_{e}, {Salient}({J}_{e}))$<br />
$[\![\mathbf{\text{doctor}}]\!]^{}_{\left\langle{}e,\left(t, t\right)\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: ({Doctor}({x}), {Human}({x}))$<br />
$[\![\mathbf{\text{saw}}]\!]^{}_{\left\langle{}e,\left\langle{}e,\left(t, t\right)\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: ({Saw}({y}, {x}), ({PhysExistence}({x}) \wedge{} {HasVisualSystem}({y})))$<br />
$[\![\mathbf{\text{the}}]\!]^{}_{\left\langle{}\left\langle{}e,\left(t, t\right)\right\rangle{},\left(e, t\right)\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,\left(t, t\right)\right\rangle{}} \: . \: (\iota{} x_{e} \: . \: (({f}({x})[{0}]) \wedge{} ({f}({x})[{1}])), \exists{}! x_{e} \: . \: (({f}({x})[{0}]) \wedge{} ({f}({x})[{1}])))$

Manually lifting all instances of type `t` up to `(t,t)` is reasonably straightforward, you just have to remember to do it.

In [2]:
%%lamb
||john2|| = J_e
||doctor2|| = L x_e : (Doctor(x), True)
||saw2|| = L x_e : L y_e : (Saw(y,x), True)

INFO (meta): Coerced guessed type for 'Doctor_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{john2}}]\!]^{}_{e} \:=\: $${J}_{e}$<br />
$[\![\mathbf{\text{doctor2}}]\!]^{}_{\left\langle{}e,\left(t, t\right)\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: ({Doctor}({x}), {True}_{t})$<br />
$[\![\mathbf{\text{saw2}}]\!]^{}_{\left\langle{}e,\left\langle{}e,\left(t, t\right)\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: ({Saw}({y}, {x}), {True}_{t})$

A version of functional application that combines lifted types:

In [3]:
def unpack(x):
    if isinstance(x, meta.Tuple):
        return (x[1], x[2])
    else:
        return (x, meta.true_term)

def paircombine(f, a):
    ftup = unpack(f)
    atup = unpack(a)
    result_left_tup = unpack(ftup[0](atup[0]).reduce_all())
    result_right = (ftup[1] & atup[1] & result_left_tup[1]).simplify_all()
    result = meta.Tuple((result_left_tup[0], result_right))
    #result = (result_left_tup[0], result_right)
    return result

# PM generalized to type `<e,(t,t)>`.  Regular PM won't do what I need.
presup_pm = %te L f_<e,(t,t)> : L g_<e,(t,t)> : L x_e : (f(x)[0] & g(x)[0], f(x)[1] & g(x)[1])

system = lang.td_system.copy()
system.add_binary_rule_uncurried(paircombine, "pair-combine")
system.remove_rule("PM")
system.add_binary_rule(presup_pm, "PresupPM", commutative=True)
system.remove_rule("FA")
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [6]:
john * doctor

CompositionResult(results=[⟦[doctor john]⟧ = (Doctor_<e,t>(J_e), (Salient_<e,t>(J_e) & Human_<e,t>(J_e)))], failures=[⟦[john doctor]⟧ = Type mismatch: '⟦john⟧ = (J_e, Salient_<e,t>(J_e))'/(e,t) and '⟦doctor⟧ = (λ x_e: (Doctor_<e,t>(x_e), Human_<e,t>(x_e)))'/<e,(t,t)> conflict (mode: Predicate Abstraction), ⟦[doctor john]⟧ = Type mismatch: '⟦doctor⟧ = (λ x_e: (Doctor_<e,t>(x_e), Human_<e,t>(x_e)))'/<e,(t,t)> and '⟦john⟧ = (J_e, Salient_<e,t>(J_e))'/(e,t) conflict (mode: Predicate Abstraction), ⟦[john doctor]⟧ = Type mismatch: 'J_e'/e and '(λ x_e: (Doctor_<e,t>(x_e), Human_<e,t>(x_e)))'/<e,(t,t)> conflict (mode: Function argument combination (unification failed)), ⟦[john doctor]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and '(J_e, Salient_<e,t>(J_e))'/(e,t) conflict (mode: Function argument combination (unification failed))])

In [7]:
(john * (saw * (the * doctor)))

CompositionResult(results=[⟦[[saw [the doctor]] john]⟧ = (Saw_<(e,e),t>(J_e, (ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))), (((ExistsExact x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e))) & Salient_<e,t>(J_e)) & (PhysExistence_<e,t>((ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))) & HasVisualSystem_<e,t>(J_e))))], failures=[⟦[john [saw [the doctor]]]⟧ = Type mismatch: '⟦john⟧ = (J_e, Salient_<e,t>(J_e))'/(e,t) and '⟦[saw [the doctor]]⟧ = ((λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))), (PhysExistence_<e,t>((ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))) & HasVisualSystem_<e,t>(y_e)))), (ExistsExact x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e))))'/(<e,(t,t)>,t) conflict (mode: Predicate Abstraction), ⟦[[saw [the doctor]] john]⟧ = Type mismatch: '⟦[saw [the doctor]]⟧ = ((λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))), (PhysExistence_<e,t>((ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))) & HasVisualSystem_<e,t>(y_e)))), (ExistsExact x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e))))'/(<e,(t,t)>,t) and '⟦john⟧ = (J_e, Salient_<e,t>(J_e))'/(e,t) conflict (mode: Predicate Abstraction), ⟦[john [saw [the doctor]]]⟧ = Type mismatch: 'J_e'/e and '(λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))), (PhysExistence_<e,t>((ι x_e: (Doctor_<e,t>(x_e) & Human_<e,t>(x_e)))) & HasVisualSystem_<e,t>(y_e))))'/<e,(t,t)> conflict (mode: Function argument combination (unification failed)), ⟦[john [saw [the doctor]]]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and '(J_e, Salient_<e,t>(J_e))'/(e,t) conflict (mode: Function argument combination (unification failed))])

In [8]:
(john2 * (saw2 * (the * doctor2)))

CompositionResult(results=[⟦[[saw2 [the doctor2]] john2]⟧ = (Saw_<(e,e),t>(J_e, (ι x_e: Doctor_<e,t>(x_e))), (ExistsExact x_e: Doctor_<e,t>(x_e)))], failures=[⟦[john2 [saw2 [the doctor2]]]⟧ = Type mismatch: '⟦john2⟧ = J_e'/e and '⟦[saw2 [the doctor2]]⟧ = ((λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: Doctor_<e,t>(x_e))), True_t)), (ExistsExact x_e: Doctor_<e,t>(x_e)))'/(<e,(t,t)>,t) conflict (mode: Predicate Abstraction), ⟦[[saw2 [the doctor2]] john2]⟧ = Type mismatch: '⟦[saw2 [the doctor2]]⟧ = ((λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: Doctor_<e,t>(x_e))), True_t)), (ExistsExact x_e: Doctor_<e,t>(x_e)))'/(<e,(t,t)>,t) and '⟦john2⟧ = J_e'/e conflict (mode: Predicate Abstraction), ⟦[john2 [saw2 [the doctor2]]]⟧ = Type mismatch: 'J_e'/e and '(λ y_e: (Saw_<(e,e),t>(y_e, (ι x_e: Doctor_<e,t>(x_e))), True_t))'/<e,(t,t)> conflict (mode: Function argument combination (unification failed)), ⟦[john2 [saw2 [the doctor2]]]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and 'J_e'/e conflict (mode: Function argument combination (unification failed))])

The above system only handles lifting of the whole type of a function or argument up to `(X,t)`.  This leads to problems with certain unlifted types:

In [9]:
%%lamb
||john3|| = J_e
||doctor3|| = L x_e : Doctor(x)
||saw3|| = L x_e : L y_e : Saw(y,x)

INFO (meta): Coerced guessed type for 'Doctor_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{john3}}]\!]^{}_{e} \:=\: $${J}_{e}$<br />
$[\![\mathbf{\text{doctor3}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Doctor}({x})$<br />
$[\![\mathbf{\text{saw3}}]\!]^{}_{\left\langle{}e,\left\langle{}e,t\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {Saw}({y}, {x})$

In [10]:
the * doctor3

CompositionResult(results=[], failures=[⟦[the doctor3]⟧ = Type mismatch: '⟦the⟧ = (λ f_<e,(t,t)>: ((ι x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n]))), (ExistsExact x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])))))'/<<e,(t,t)>,(e,t)> and '⟦doctor3⟧ = (λ x_e: Doctor_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[doctor3 the]⟧ = Type mismatch: '⟦doctor3⟧ = (λ x_e: Doctor_<e,t>(x_e))'/<e,t> and '⟦the⟧ = (λ f_<e,(t,t)>: ((ι x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n]))), (ExistsExact x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])))))'/<<e,(t,t)>,(e,t)> conflict (mode: Predicate Abstraction), ⟦[the doctor3]⟧ = Type mismatch: '(λ f_<e,(t,t)>: ((ι x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n]))), (ExistsExact x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])))))'/<<e,(t,t)>,(e,t)> and '(λ x_e: Doctor_<e,t>(x_e))'/<e,t> conflict (mode: Function argument combination (unification failed)), ⟦[doctor3 the]⟧ = Type mismatch: '(λ x_e: Doctor_<e,t>(x_e))'/<e,t> and '(λ f_<e,(t,t)>: ((ι x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n]))), (ExistsExact x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])))))'/<<e,(t,t)>,(e,t)> conflict (mode: Function argument combination (unification failed)), ⟦[the doctor3]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and '(λ f_<e,(t,t)>: ((ι x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n]))), (ExistsExact x_e: ((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])))))'/<<e,(t,t)>,(e,t)> conflict (mode: Function argument combination (unification failed))])

To deal with this, I will define a variant of $\lambda$ that is guaranteed to return a tuple type `(X,t)`.  Otherwise this is just an ordinary $\lambda$ binder.

In [11]:
class PLFun(meta.LFun):
    canonical_name = "PLambda"
    secondary_names = {"PL", "Pλ", "plambda"}
    op_name_uni="λ"
    op_name_latex="\\lambda{}"

    def __init__(self, var_or_vtype, body, varname=None, let=False, assignment=None, type_check=True):
        if not (isinstance(body.type, types.TupleType) and len(body.type) == 2 and body.type[1] == types.type_t):
            body = meta.Tuple((body, meta.true_term))
        super().__init__(var_or_vtype=var_or_vtype, body=body, varname=varname, assignment=assignment, type_check=type_check)
        
meta.BindingOp.add_op(PLFun)

In [12]:
%%lamb
||doctor4|| = PL x_e : Doctor(x)
||saw4|| = PL x_e : PL y_e : Saw(y,x)

INFO (meta): Coerced guessed type for 'Doctor_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Saw_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{doctor4}}]\!]^{}_{\left\langle{}e,\left(t, t\right)\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: ({Doctor}({x}), {True}_{t})$<br />
$[\![\mathbf{\text{saw4}}]\!]^{}_{\left\langle{}e,\left(\left\langle{}e,\left(t, t\right)\right\rangle{}, t\right)\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: (\lambda{} y_{e} \: . \: ({Saw}({y}, {x}), {True}_{t}), {True}_{t})$

In [ ]:
john * (saw4 * (the * doctor4))

### A complicated case

One of the tricky things about presuppositional grammars is that you have to be completely explicit about all sorts of cases where the data is actually not that clear.  One such family of cases is any item that binds variables, such as quantificational determiners.  The reason you need to be explicit is that such operators may bind into presuppositions.  For example, (1) instantiates a fairly complicated case:

    (1) Every student that likes his brother likes his sister.

Here are some plausible inferences from (1):

* Every student is male.
* If a student has a brother, that student has a sister.
* Every student has a brother. (?)
* If a student likes his brother, that student has a sister. (?)
* Every student has a sister. (??)

It is _extremely_ hard to get all of these, but also somewhat unclear if one should.  The following entry for `every` gets the first two.  The fourth could be gotten by adding `Forall x_e : f(x)[0] ==> g(x)[1]` as a conjunct to the presupposition, but it requires a version of Iota that works in a presupposition.  The third and fifth, if desireable, need a completely different strategy (further modification of the PM rule).

In [13]:
%%lamb
||every|| = L f_<e,(t,t)> : L g_<e,(t,t)> : ((Forall x_e: (f(x)[0] & f(x)[1]) ==> g(x)[0]), (Forall x_e: (f(x)[1]) ==> g(x)[1]))

$[\![\mathbf{\text{every}}]\!]^{}_{\left\langle{}\left\langle{}e,\left(t, t\right)\right\rangle{},\left\langle{}\left\langle{}e,\left(t, t\right)\right\rangle{},\left(t, t\right)\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,\left(t, t\right)\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,\left(t, t\right)\right\rangle{}} \: . \: (\forall{} x_{e} \: . \: ((({f}({x})[{0}]) \wedge{} ({f}({x})[{1}])) \rightarrow{} ({g}({x})[{0}])), \forall{} x_{e} \: . \: (({f}({x})[{1}]) \rightarrow{} ({g}({x})[{1}])))$

In [14]:
class PresupPronoun(lang.IndexedPronoun):
    def __init__(self, name, condition, index, typ=None):
        super().__init__(name, index, typ=typ)
        var = self.content
        self.content = meta.Tuple((var, condition(var).reduce_all()))

male_prop = %te L x_e : Male_<e,t>(x)
him = PresupPronoun("him", male_prop, 5)
him

⟦him5⟧ = (var5_e, Male_<e,t>(var5_e))

In [15]:
%%lamb
||student|| = PL x_e : Student(x)
||likes|| = L x_e : PL y_e : Likes(y,x)
||brother|| = L x_e : PL y_e : BrotherOf(y,x)
||sister|| = L x_e : PL y_e : SisterOf(y,x)
||POSS|| = L f_<e,<e,(t,t)>> : L x_e : ((Iota y_e : f(x)(y)[0] & f(x)(y)[1]), (ExistsExact y : f(x)(y)[0] & f(x)(y)[1]))

INFO (meta): Coerced guessed type for 'Student_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Likes_t' into <(e,e),t>, to match argument '(y_e, x_e)'
INFO (meta): Coerced guessed type for 'BrotherOf_t' into <(e,e),t>, to match argument '(y_e, x_e)'
INFO (meta): Coerced guessed type for 'SisterOf_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{student}}]\!]^{}_{\left\langle{}e,\left(t, t\right)\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: ({Student}({x}), {True}_{t})$<br />
$[\![\mathbf{\text{likes}}]\!]^{}_{\left\langle{}e,\left\langle{}e,\left(t, t\right)\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: ({Likes}({y}, {x}), {True}_{t})$<br />
$[\![\mathbf{\text{brother}}]\!]^{}_{\left\langle{}e,\left\langle{}e,\left(t, t\right)\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: ({BrotherOf}({y}, {x}), {True}_{t})$<br />
$[\![\mathbf{\text{sister}}]\!]^{}_{\left\langle{}e,\left\langle{}e,\left(t, t\right)\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: ({SisterOf}({y}, {x}), {True}_{t})$<br />
$[\![\mathbf{\text{POSS}}]\!]^{}_{\left\langle{}\left\langle{}e,\left\langle{}e,\left(t, t\right)\right\rangle{}\right\rangle{},\left\langle{}e,\left(e, t\right)\right\rangle{}\right\rangle{}} \:=\: $$\lambda{} f_{\left\langle{}e,\left\langle{}e,\left(t, t\right)\right\rangle{}\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: (\iota{} y_{e} \: . \: (({f}({x})({y})[{0}]) \wedge{} ({f}({x})({y})[{1}])), \exists{}! y_{e} \: . \: (({f}({x})({y})[{0}]) \wedge{} ({f}({x})({y})[{1}])))$

In [16]:
john * (POSS * brother)

CompositionResult(results=[⟦[[POSS brother] john]⟧ = ((ι y_e: BrotherOf_<(e,e),t>(y_e, J_e)), (Salient_<e,t>(J_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, J_e))))], failures=[⟦[john [POSS brother]]⟧ = Type mismatch: '⟦john⟧ = (J_e, Salient_<e,t>(J_e))'/(e,t) and '⟦[POSS brother]⟧ = ((λ x_e: ((ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)), (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))), True_t)'/(<e,(e,t)>,t) conflict (mode: Predicate Abstraction), ⟦[[POSS brother] john]⟧ = Type mismatch: '⟦[POSS brother]⟧ = ((λ x_e: ((ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)), (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))), True_t)'/(<e,(e,t)>,t) and '⟦john⟧ = (J_e, Salient_<e,t>(J_e))'/(e,t) conflict (mode: Predicate Abstraction), ⟦[john [POSS brother]]⟧ = Type mismatch: 'J_e'/e and '(λ x_e: ((ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)), (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))))'/<e,(e,t)> conflict (mode: Function argument combination (unification failed)), ⟦[john [POSS brother]]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and '(J_e, Salient_<e,t>(J_e))'/(e,t) conflict (mode: Function argument combination (unification failed))])

In [17]:
him * (POSS * brother)

CompositionResult(results=[⟦[[POSS brother] him5]⟧ = ((ι y_e: BrotherOf_<(e,e),t>(y_e, var5_e)), (Male_<e,t>(var5_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, var5_e))))], failures=[⟦[him5 [POSS brother]]⟧ = Type mismatch: '⟦him5⟧ = (var5_e, Male_<e,t>(var5_e))'/(e,t) and '⟦[POSS brother]⟧ = ((λ x_e: ((ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)), (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))), True_t)'/(<e,(e,t)>,t) conflict (mode: Predicate Abstraction), ⟦[[POSS brother] him5]⟧ = Type mismatch: '⟦[POSS brother]⟧ = ((λ x_e: ((ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)), (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))), True_t)'/(<e,(e,t)>,t) and '⟦him5⟧ = (var5_e, Male_<e,t>(var5_e))'/(e,t) conflict (mode: Predicate Abstraction), ⟦[him5 [POSS brother]]⟧ = Type mismatch: 'var5_e'/e and '(λ x_e: ((ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)), (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))))'/<e,(e,t)> conflict (mode: Function argument combination (unification failed)), ⟦[him5 [POSS brother]]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and '(var5_e, Male_<e,t>(var5_e))'/(e,t) conflict (mode: Function argument combination (unification failed))])

In [18]:
# compute "[every student [5 [that t_5 likes his_5 brother]]]"
subject = (every * (student * (lang.Binder(5) * ((lang.Trace(5) * (likes * (PresupPronoun("him", male_prop, 5) * (POSS * brother))))))))
subject

CompositionResult(results=[⟦[every [student [5 [[likes [[POSS brother] him5]] t5]]]]⟧ = ((λ g_<e,(t,t)>: ((Forall x_e: (((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) & (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))) >> (g_<e,(t,t)>(x_e)[1_n]))))), True_t)], failures=[⟦[every [student [5 [[likes [[POSS brother] him5]] t5]]]]⟧ = Type mismatch: '⟦every⟧ = (λ f_<e,(t,t)>: (λ g_<e,(t,t)>: ((Forall x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((f_<e,(t,t)>(x_e)[1_n]) >> (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,(t,t)>> and '⟦[student [5 [[likes [[POSS brother] him5]] t5]]]⟧ = (λ x_e: ((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))), (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))))'/<e,(t,t)> conflict (mode: Predicate Abstraction), ⟦[[student [5 [[likes [[POSS brother] him5]] t5]]] every]⟧ = Type mismatch: '⟦[student [5 [[likes [[POSS brother] him5]] t5]]]⟧ = (λ x_e: ((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))), (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))))'/<e,(t,t)> and '⟦every⟧ = (λ f_<e,(t,t)>: (λ g_<e,(t,t)>: ((Forall x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((f_<e,(t,t)>(x_e)[1_n]) >> (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,(t,t)>> conflict (mode: Predicate Abstraction), ⟦[[student [5 [[likes [[POSS brother] him5]] t5]]] every]⟧ = Type mismatch: '(λ x_e: ((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))), (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))))'/<e,(t,t)> and '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: ((Forall x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((f_<e,(t,t)>(x_e)[1_n]) >> (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,(t,t)>> conflict (mode: Function argument combination (unification failed)), ⟦[every [student [5 [[likes [[POSS brother] him5]] t5]]]]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: ((Forall x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (f_<e,(t,t)>(x_e)[1_n])) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((f_<e,(t,t)>(x_e)[1_n]) >> (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,(t,t)>> conflict (mode: Function argument combination (unification failed))])

In [19]:
# assume that the subject has QR'd in order to bind the possessive pronoun
subject * (lang.Binder(6) * (lang.Trace(6) * (likes * (PresupPronoun("him", male_prop, 6) * (POSS * sister)))))

CompositionResult(results=[⟦[[every [student [5 [[likes [[POSS brother] him5]] t5]]]] [6 [[likes [[POSS sister] him6]] t6]]]⟧ = ((Forall x_e: (((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) & (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) >> Likes_<(e,e),t>(x_e, (ι y_e: SisterOf_<(e,e),t>(y_e, x_e))))), (Forall x_e: ((Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))) >> (Male_<e,t>(x_e) & (ExistsExact y_e: SisterOf_<(e,e),t>(y_e, x_e))))))], failures=[⟦[[every [student [5 [[likes [[POSS brother] him5]] t5]]]] [6 [[likes [[POSS sister] him6]] t6]]]⟧ = Type mismatch: '⟦[every [student [5 [[likes [[POSS brother] him5]] t5]]]]⟧ = ((λ g_<e,(t,t)>: ((Forall x_e: (((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) & (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))) >> (g_<e,(t,t)>(x_e)[1_n]))))), True_t)'/(<<e,(t,t)>,(t,t)>,t) and '⟦[6 [[likes [[POSS sister] him6]] t6]]⟧ = (λ x_e: (Likes_<(e,e),t>(x_e, (ι y_e: SisterOf_<(e,e),t>(y_e, x_e))), (Male_<e,t>(x_e) & (ExistsExact y_e: SisterOf_<(e,e),t>(y_e, x_e)))))'/<e,(t,t)> conflict (mode: Predicate Abstraction), ⟦[[6 [[likes [[POSS sister] him6]] t6]] [every [student [5 [[likes [[POSS brother] him5]] t5]]]]]⟧ = Type mismatch: '⟦[6 [[likes [[POSS sister] him6]] t6]]⟧ = (λ x_e: (Likes_<(e,e),t>(x_e, (ι y_e: SisterOf_<(e,e),t>(y_e, x_e))), (Male_<e,t>(x_e) & (ExistsExact y_e: SisterOf_<(e,e),t>(y_e, x_e)))))'/<e,(t,t)> and '⟦[every [student [5 [[likes [[POSS brother] him5]] t5]]]]⟧ = ((λ g_<e,(t,t)>: ((Forall x_e: (((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) & (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))) >> (g_<e,(t,t)>(x_e)[1_n]))))), True_t)'/(<<e,(t,t)>,(t,t)>,t) conflict (mode: Predicate Abstraction), ⟦[[6 [[likes [[POSS sister] him6]] t6]] [every [student [5 [[likes [[POSS brother] him5]] t5]]]]]⟧ = Type mismatch: '(λ x_e: (Likes_<(e,e),t>(x_e, (ι y_e: SisterOf_<(e,e),t>(y_e, x_e))), (Male_<e,t>(x_e) & (ExistsExact y_e: SisterOf_<(e,e),t>(y_e, x_e)))))'/<e,(t,t)> and '(λ g_<e,(t,t)>: ((Forall x_e: (((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) & (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))) >> (g_<e,(t,t)>(x_e)[1_n])))))'/<<e,(t,t)>,(t,t)> conflict (mode: Function argument combination (unification failed)), ⟦[[every [student [5 [[likes [[POSS brother] him5]] t5]]]] [6 [[likes [[POSS sister] him6]] t6]]]⟧ = Type mismatch: '(λ f_<e,(t,t)>: (λ g_<e,(t,t)>: (λ x_e: (((f_<e,(t,t)>(x_e)[0_n]) & (g_<e,(t,t)>(x_e)[0_n])), ((f_<e,(t,t)>(x_e)[1_n]) & (g_<e,(t,t)>(x_e)[1_n]))))))'/<<e,(t,t)>,<<e,(t,t)>,<e,(t,t)>>> and '((λ g_<e,(t,t)>: ((Forall x_e: (((Student_<e,t>(x_e) & Likes_<(e,e),t>(x_e, (ι y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) & (Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e)))) >> (g_<e,(t,t)>(x_e)[0_n]))), (Forall x_e: ((Male_<e,t>(x_e) & (ExistsExact y_e: BrotherOf_<(e,e),t>(y_e, x_e))) >> (g_<e,(t,t)>(x_e)[1_n]))))), True_t)'/(<<e,(t,t)>,(t,t)>,t) conflict (mode: Function argument combination (unification failed))])